In [2]:
# ============================================================
# 단계 1: 데이터 적재 및 전처리 파이프라인 구축
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

print("=" * 70)
print("단계 1: 데이터 적재 및 전처리 파이프라인")
print("=" * 70)

# 1.1 데이터 로드
df = pd.read_csv('train.csv')

print(f"\n데이터 크기: {df.shape}")
print(f"컬럼 수: {len(df.columns)}")

# 1.2 기본 정보 확인
print(f"\n데이터 타입:")
print(df.dtypes.value_counts())

print(f"\n결측치 확인:")
print(f"전체 결측치: {df.isna().sum().sum()}개")

print(f"\n라벨 분포:")
print(df['label'].value_counts(normalize=True))

# 1.3 컬럼 그룹 정의
fp_cols = [col for col in df.columns if col.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']
id_col = 'SMILES'
label_col = 'label'

print(f"\nFingerprint 컬럼: {len(fp_cols)}개")
print(f"물성 컬럼: {desc_cols}")

# 1.4 X, y 분리
X = df.drop(columns=[label_col])
y = df[label_col].astype(int)

# 1.5 전처리 파이프라인 구성
fp_transformer = SimpleImputer(strategy='constant', fill_value=0)
# - 목적: Fingerprint는 0/1 값을 가지는 binary 특성이므로
#         결측치를 0으로 대치 (분자 구조에 해당 substructure 없음을 의미)
# - strategy='constant': 고정된 상수값으로 채움
# - fill_value=0: 결측치를 모두 0으로 대치

desc_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # 1단계: 중앙값 대치
    # - 1단계: 결측치 처리
    # - strategy='median': 각 컬럼의 중앙값으로 결측치 대치
    # - 평균 대신 중앙값 사용 이유: 이상치(outlier)의 영향을 덜 받음
    ('scaler', StandardScaler()) # 2단계: 표준화
    # - 2단계: 특성 스케일링 (표준화)
    # - 각 컬럼을 평균 0, 표준편차 1로 변환
    # - 공식: z = (x - μ) / σ
    # - 이유: MolWt(분자량), clogP, sa_score, qed는
    #         서로 다른 스케일을 가지므로 정규화 필요
])

preprocessor = ColumnTransformer(
    transformers=[
        ('fp', fp_transformer, fp_cols),
        ('desc', desc_transformer, desc_cols)
    ],
    remainder='drop'
)

# 1.6 교차검증 분할 테스트
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"\n전처리 파이프라인 구축 완료")
print(f"  - Fingerprint: 결측 → 0 대치")
print(f"  - 물성: 결측 → 중앙값 대치 + StandardScaler")
print(f"  - 교차검증: 5-Fold Stratified")

# 1.7 샘플 변환 테스트
for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    print(f"\nFold {fold}: 학습 {Xt_tr.shape}, 검증 {Xt_va.shape}")
    break  # 첫 fold만 확인

print("\n✓ 단계 1 완료")


단계 1: 데이터 적재 및 전처리 파이프라인

데이터 크기: (8349, 3078)
컬럼 수: 3078

데이터 타입:
int64      3073
float64       4
object        1
Name: count, dtype: int64

결측치 확인:
전체 결측치: 0개

라벨 분포:
label
1    0.544017
0    0.455983
Name: proportion, dtype: float64

Fingerprint 컬럼: 3072개
물성 컬럼: ['MolWt', 'clogp', 'sa_score', 'qed']

전처리 파이프라인 구축 완료
  - Fingerprint: 결측 → 0 대치
  - 물성: 결측 → 중앙값 대치 + StandardScaler
  - 교차검증: 5-Fold Stratified

Fold 1: 학습 (6679, 3076), 검증 (1670, 3076)

✓ 단계 1 완료


In [5]:
import lightgbm as lgb

print("\n" + "=" * 70)
print("단계 2: 베이스라인 모델 학습 (개선)")
print("=" * 70)

RANDOM_STATE = 42
f1_scores = []
models = []

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    # 전처리
    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    # 모델 학습 (개선된 파라미터)
    model = LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=7,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1.0,
        reg_lambda=1.0,
        class_weight='balanced',
        random_state=RANDOM_STATE,
        verbose=-1
    )

    model.fit(
        Xt_tr, y_tr,
        eval_set=[(Xt_va, y_va)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )

    models.append(model)

    # 예측 및 평가
    y_pred = model.predict(Xt_va)
    f1 = f1_score(y_va, y_pred)
    f1_scores.append(f1)

    print(f"\n[Fold {fold}]")
    print(f"  학습: {Xt_tr.shape}, 검증: {Xt_va.shape}")
    print(f"  최적 Iteration: {model.best_iteration_}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  분류 리포트:")
    print(classification_report(y_va, y_pred,
                                target_names=['독성 有(0)', '독성 無(1)']))
    print(f"  혼동 행렬:")
    print(confusion_matrix(y_va, y_pred))

print(f"\n{'='*70}")
print(f"베이스라인 평균 F1 Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"Fold별 F1: {[f'{f:.4f}' for f in f1_scores]}")
print("\n✓ 단계 2 완료")



단계 2: 베이스라인 모델 학습 (개선)
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.406499


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



[Fold 1]
  학습: (6679, 3076), 검증: (1670, 3076)
  최적 Iteration: 500
  F1 Score: 0.8256
  분류 리포트:
              precision    recall  f1-score   support

      비활성(0)       0.79      0.81      0.80       761
       활성(1)       0.83      0.82      0.83       909

    accuracy                           0.81      1670
   macro avg       0.81      0.81      0.81      1670
weighted avg       0.81      0.81      0.81      1670

  혼동 행렬:
[[613 148]
 [166 743]]
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[469]	valid_0's binary_logloss: 0.443228


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



[Fold 2]
  학습: (6679, 3076), 검증: (1670, 3076)
  최적 Iteration: 469
  F1 Score: 0.8011
  분류 리포트:
              precision    recall  f1-score   support

      비활성(0)       0.76      0.77      0.76       761
       활성(1)       0.80      0.80      0.80       909

    accuracy                           0.78      1670
   macro avg       0.78      0.78      0.78      1670
weighted avg       0.78      0.78      0.78      1670

  혼동 행렬:
[[585 176]
 [184 725]]
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[451]	valid_0's binary_logloss: 0.460899

[Fold 3]
  학습: (6679, 3076), 검증: (1670, 3076)
  최적 Iteration: 451
  F1 Score: 0.7871
  분류 리포트:


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

      비활성(0)       0.74      0.78      0.76       762
       활성(1)       0.81      0.77      0.79       908

    accuracy                           0.77      1670
   macro avg       0.77      0.77      0.77      1670
weighted avg       0.78      0.77      0.77      1670

  혼동 행렬:
[[596 166]
 [211 697]]
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[498]	valid_0's binary_logloss: 0.419175


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



[Fold 4]
  학습: (6679, 3076), 검증: (1670, 3076)
  최적 Iteration: 498
  F1 Score: 0.8233
  분류 리포트:
              precision    recall  f1-score   support

      비활성(0)       0.79      0.78      0.79       762
       활성(1)       0.82      0.83      0.82       908

    accuracy                           0.81      1670
   macro avg       0.81      0.81      0.81      1670
weighted avg       0.81      0.81      0.81      1670

  혼동 행렬:
[[598 164]
 [158 750]]
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.415413

[Fold 5]
  학습: (6680, 3076), 검증: (1669, 3076)
  최적 Iteration: 500
  F1 Score: 0.8240
  분류 리포트:
              precision    recall  f1-score   support

      비활성(0)       0.79      0.79      0.79       761
       활성(1)       0.82      0.82      0.82       908

    accuracy                           0.81      1669
   macro avg       0.81      0.81      0.81      1669
weighted avg       0.81    

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [7]:
import shap
import matplotlib.pyplot as plt

print("\n" + "=" * 70)
print("단계 3: 특징 중요도 분석 (개선)")
print("=" * 70)

# 컬럼 그룹 정의
fp_cols = [col for col in df.columns if col.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']
label_col = 'label'

# ⭐ 중요: feature_names 정의 (전처리 후 순서)
feature_names = fp_cols + desc_cols  # Fingerprint 컬럼 + 물성 컬럼

print(f"전체 피처 수: {len(feature_names)}개")
print(f"  - Fingerprint: {len(fp_cols)}개")
print(f"  - 물성: {len(desc_cols)}개")

# X, y 분리
X = df.drop(columns=[label_col])
y = df[label_col].astype(int)

# 전처리 파이프라인
preprocessor = ColumnTransformer(
    transformers=[
        ('fp', SimpleImputer(strategy='constant', fill_value=0), fp_cols),
        ('desc', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), desc_cols)
    ],
    remainder='drop'
)

# 교차검증 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# CV 기반 중요도 계산
fold_importances = []
for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr = X.iloc[tr_idx]
    y_tr = y.iloc[tr_idx]

    Xt_tr = preprocessor.fit_transform(X_tr)

    model = LGBMClassifier(
        n_estimators=500, learning_rate=0.05, max_depth=7,
        num_leaves=31, class_weight='balanced',
        random_state=42, verbose=-1
    )
    model.fit(Xt_tr, y_tr)

    # Gain 기반 중요도
    importances = model.booster_.feature_importance(importance_type='gain')
    fold_importances.append(importances)

# 통계 계산
importance_mean = np.mean(fold_importances, axis=0)
importance_std = np.std(fold_importances, axis=0)

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance_mean': importance_mean,
    'importance_std': importance_std
}).sort_values('importance_mean', ascending=False)

print("\n=== 상위 20개 중요 피처 (Gain 기반) ===")
print(importance_df.head(20).to_string(index=False))

# 물성 피처 분석
print("\n=== 물성 피처 중요도 ===")
desc_importance = importance_df[importance_df['feature'].isin(desc_cols)]
print(desc_importance.to_string(index=False))

# 저중요도 피처 필터링
threshold = importance_df['importance_mean'].sum() * 0.01
selected_features = importance_df[importance_df['importance_mean'] >= threshold]['feature'].tolist()
print(f"\n선택된 피처: {len(selected_features)}개 (임계값: {threshold:.2f})")

# 시각화
fig, ax = plt.subplots(figsize=(10, 8))
top20 = importance_df.head(20)
ax.barh(range(len(top20)), top20['importance_mean'])
ax.set_yticks(range(len(top20)))
ax.set_yticklabels(top20['feature'])
ax.invert_yaxis()
ax.set_xlabel('Importance (Gain)')
ax.set_title('Top 20 Feature Importance')
plt.tight_layout()
plt.savefig('feature_importance_top20.png', dpi=300, bbox_inches='tight')
plt.close()

# CSV 저장
importance_df.to_csv('feature_importance_cv.csv', index=False)
print("\n✓ 특징 중요도 저장: feature_importance_cv.csv")
print("✓ 시각화 저장: feature_importance_top20.png")
print("✓ 단계 3 완료")



단계 3: 특징 중요도 분석 (개선)
전체 피처 수: 3076개
  - Fingerprint: 3072개
  - 물성: 4개

=== 상위 20개 중요 피처 (Gain 기반) ===
  feature  importance_mean  importance_std
    clogp      9086.245138      329.690053
 ecfp_807      2481.696333      193.827500
 sa_score      1437.898631       95.288599
      qed      1260.252706       66.139287
 fcfp_926      1254.523910      243.868331
  fcfp_18      1140.655177       41.684327
    MolWt      1108.381560       97.641585
 ecfp_887       554.562280      127.046617
 ecfp_219       502.327773      155.912626
 ecfp_767       491.163915      239.430468
 ecfp_893       403.326927      237.587663
 fcfp_546       394.383086      139.921297
 fcfp_671       382.294291       77.640999
 fcfp_370       343.620895       74.114468
 ptfp_666       306.139495       53.843023
ptfp_1013       295.335594       78.396048
 ptfp_596       265.862897       51.307877
fcfp_1008       263.407184       53.722803
 ptfp_281       222.196898       94.925660
 fcfp_968       219.665339       43.0

In [8]:
# ============================================================
# 단계 4: 피처 선택 + 임계값 최적화 (통합 버전)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score, precision_recall_curve, roc_curve,
    roc_auc_score, average_precision_score,
    classification_report, confusion_matrix
)
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

print("=" * 70)
print("단계 4: 피처 선택 기반 임계값 최적화")
print("=" * 70)

# ============================================================
# 1. 데이터 로드 및 피처 선택
# ============================================================

# 특징 중요도 로드
importance_df = pd.read_csv('feature_importance_cv.csv')

# 상위 150개 피처 선택 (실험적으로 조정 가능: 100~200)
TOP_N = 150
selected_features = importance_df.head(TOP_N)['feature'].tolist()

print(f"\n[피처 선택]")
print(f"  전체 피처: {len(importance_df)}개")
print(f"  선택된 피처: {len(selected_features)}개")
print(f"  제거된 피처: {len(importance_df) - len(selected_features)}개")

# 물성 피처 확인
desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']
desc_in_selected = [f for f in selected_features if f in desc_cols]
print(f"  선택된 물성 피처: {desc_in_selected}")

# 데이터 로드
df = pd.read_csv('train.csv')
X = df[selected_features]
y = df['label'].astype(int)

print(f"\n데이터 크기: {X.shape}")
print(f"레이블 분포: {y.value_counts().to_dict()}")

# ============================================================
# 2. 전처리 파이프라인 (선택된 피처 기반)
# ============================================================

fp_cols_selected = [f for f in selected_features if f.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
desc_cols_selected = [f for f in selected_features if f in desc_cols]

print(f"\n전처리 대상:")
print(f"  Fingerprint: {len(fp_cols_selected)}개")
print(f"  물성: {len(desc_cols_selected)}개")

preprocessor = ColumnTransformer(
    transformers=[
        ('fp', SimpleImputer(strategy='constant', fill_value=0), fp_cols_selected),
        ('desc', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), desc_cols_selected)
    ],
    remainder='drop'
)

# ============================================================
# 3. 교차검증 기반 임계값 최적화
# ============================================================

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

f1_scores_baseline = []  # 기본 임계값 (0.5)
f1_scores_optimized = []  # 최적화된 임계값
best_thresholds = []
fold_results = []

print("\n" + "=" * 70)
print("교차검증 시작")
print("=" * 70)

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    # 전처리
    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    # 모델 학습 (개선된 하이퍼파라미터)
    model = LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=7,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1.0,
        reg_lambda=1.0,
        class_weight='balanced',
        random_state=42,
        verbose=-1
    )

    model.fit(Xt_tr, y_tr)

    # 확률 예측
    y_proba = model.predict_proba(Xt_va)[:, 1]

    # --------------------------------------------------------
    # 3-1. 임계값 그리드 탐색 (F1 최대화)
    # --------------------------------------------------------
    thresholds_grid = np.linspace(0.1, 0.9, 81)
    f1_scores_grid = []

    for thresh in thresholds_grid:
        y_pred = (y_proba >= thresh).astype(int)
        f1_scores_grid.append(f1_score(y_va, y_pred))

    best_idx = np.argmax(f1_scores_grid)
    best_threshold = thresholds_grid[best_idx]
    best_f1 = f1_scores_grid[best_idx]
    best_thresholds.append(best_threshold)

    # --------------------------------------------------------
    # 3-2. 기본 임계값 vs 최적 임계값 비교
    # --------------------------------------------------------
    y_pred_baseline = (y_proba >= 0.5).astype(int)
    y_pred_optimized = (y_proba >= best_threshold).astype(int)

    f1_baseline = f1_score(y_va, y_pred_baseline)
    f1_optimized = f1_score(y_va, y_pred_optimized)

    f1_scores_baseline.append(f1_baseline)
    f1_scores_optimized.append(f1_optimized)

    # --------------------------------------------------------
    # 3-3. 추가 메트릭 계산
    # --------------------------------------------------------
    roc_auc = roc_auc_score(y_va, y_proba)
    pr_auc = average_precision_score(y_va, y_proba)

    # Confidence 분석
    confidence = np.abs(y_proba - 0.5)
    low_confidence_count = (confidence < 0.1).sum()
    avg_confidence = confidence.mean()

    # --------------------------------------------------------
    # 3-4. 결과 출력
    # --------------------------------------------------------
    print(f"\n{'='*70}")
    print(f"[Fold {fold}]")
    print(f"{'='*70}")
    print(f"피처: {len(selected_features)}개 사용")
    print(f"학습 데이터: {Xt_tr.shape}, 검증 데이터: {Xt_va.shape}")
    print(f"\n[성능 비교]")
    print(f"  기본 임계값 (0.50): F1 = {f1_baseline:.4f}")
    print(f"  최적 임계값 ({best_threshold:.3f}): F1 = {f1_optimized:.4f}")
    print(f"  개선량: {f1_optimized - f1_baseline:+.4f}")
    print(f"\n[추가 메트릭]")
    print(f"  ROC-AUC: {roc_auc:.4f}")
    print(f"  PR-AUC: {pr_auc:.4f}")
    print(f"  평균 Confidence: {avg_confidence:.4f}")
    print(f"  낮은 Confidence (<0.1): {low_confidence_count}개 ({low_confidence_count/len(y_va)*100:.1f}%)")

    print(f"\n[혼동행렬 - 최적 임계값]")
    cm = confusion_matrix(y_va, y_pred_optimized)
    print(cm)
    tn, fp, fn, tp = cm.ravel()
    print(f"  TN={tn}, FP={fp}, FN={fn}, TP={tp}")
    print(f"  FPR={fp/(fp+tn):.3f}, FNR={fn/(fn+tp):.3f}")

    print(f"\n[분류 리포트 - 최적 임계값]")
    print(classification_report(y_va, y_pred_optimized,
                                target_names=['비활성(0)', '활성(1)'],
                                digits=4))

    # 결과 저장
    fold_results.append({
        'fold': fold,
        'threshold': best_threshold,
        'f1_baseline': f1_baseline,
        'f1_optimized': f1_optimized,
        'improvement': f1_optimized - f1_baseline,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'avg_confidence': avg_confidence,
        'low_confidence_pct': low_confidence_count/len(y_va)*100
    })

    # --------------------------------------------------------
    # 3-5. 시각화 (첫 번째 Fold만)
    # --------------------------------------------------------
    if fold == 1:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        # 1) F1 Score vs Threshold
        axes[0, 0].plot(thresholds_grid, f1_scores_grid, marker='o', markersize=3)
        axes[0, 0].axvline(x=best_threshold, color='r', linestyle='--',
                          label=f'Best F1={best_f1:.4f} at {best_threshold:.3f}')
        axes[0, 0].axvline(x=0.5, color='gray', linestyle=':', alpha=0.5, label='Default (0.5)')
        axes[0, 0].set_xlabel('Threshold')
        axes[0, 0].set_ylabel('F1 Score')
        axes[0, 0].set_title('F1 Score vs Threshold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # 2) Precision-Recall Curve
        precision, recall, pr_thresholds = precision_recall_curve(y_va, y_proba)
        axes[0, 1].plot(recall, precision, marker='.', markersize=2)
        axes[0, 1].set_xlabel('Recall')
        axes[0, 1].set_ylabel('Precision')
        axes[0, 1].set_title(f'Precision-Recall Curve (AP={pr_auc:.4f})')
        axes[0, 1].grid(True, alpha=0.3)

        # 3) ROC Curve
        fpr, tpr, roc_thresholds = roc_curve(y_va, y_proba)
        axes[1, 0].plot(fpr, tpr, marker='.', markersize=2)
        axes[1, 0].plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random')
        axes[1, 0].set_xlabel('False Positive Rate')
        axes[1, 0].set_ylabel('True Positive Rate')
        axes[1, 0].set_title(f'ROC Curve (AUC={roc_auc:.4f})')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

        # 4) Confidence Distribution
        axes[1, 1].hist(confidence, bins=50, edgecolor='black', alpha=0.7)
        axes[1, 1].axvline(x=0.1, color='r', linestyle='--',
                          label=f'Low Confidence: {low_confidence_count}개')
        axes[1, 1].set_xlabel('Confidence (|prob - 0.5|)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Confidence Distribution')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('threshold_optimization_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()
        print(f"\n✓ 시각화 저장: threshold_optimization_analysis.png")

# ============================================================
# 4. 최종 통계 및 결과 저장
# ============================================================

print("\n" + "=" * 70)
print("최종 통계")
print("=" * 70)

print(f"\n[피처 선택 효과]")
print(f"  사용 피처: {len(selected_features)}개")
print(f"  제거 피처: {len(importance_df) - len(selected_features)}개")

print(f"\n[기본 임계값 (0.5)]")
print(f"  평균 F1: {np.mean(f1_scores_baseline):.4f}")
print(f"  표준편차: {np.std(f1_scores_baseline):.4f}")
print(f"  Fold별: {[f'{f:.4f}' for f in f1_scores_baseline]}")

print(f"\n[최적화된 임계값]")
print(f"  평균 F1: {np.mean(f1_scores_optimized):.4f}")
print(f"  표준편차: {np.std(f1_scores_optimized):.4f}")
print(f"  Fold별: {[f'{f:.4f}' for f in f1_scores_optimized]}")

print(f"\n[임계값 통계]")
print(f"  평균 임계값: {np.mean(best_thresholds):.3f}")
print(f"  표준편차: {np.std(best_thresholds):.3f}")
print(f"  범위: [{np.min(best_thresholds):.3f}, {np.max(best_thresholds):.3f}]")
print(f"  Fold별: {[f'{t:.3f}' for t in best_thresholds]}")

# 임계값 안정성 평가
threshold_cv = np.std(best_thresholds) / (np.mean(best_thresholds) + 1e-10)
print(f"  변동계수: {threshold_cv:.3f}")

if threshold_cv < 0.10:
    print("  ✓ 임계값이 매우 안정적입니다.")
elif threshold_cv < 0.20:
    print("  ✓ 임계값이 안정적입니다.")
else:
    print("  ⚠️ 경고: 임계값 변동이 큽니다. 모델 재검토 필요.")

print(f"\n[전체 개선량]")
improvement = np.mean(f1_scores_optimized) - np.mean(f1_scores_baseline)
print(f"  F1 개선: {improvement:+.4f} ({improvement/np.mean(f1_scores_baseline)*100:+.2f}%)")

# 결과 DataFrame 저장
results_df = pd.DataFrame(fold_results)
results_df.to_csv('threshold_optimization_results_v2.csv', index=False)

print(f"\n✓ 결과 저장: threshold_optimization_results_v2.csv")

# 최종 권장 임계값
optimal_threshold_final = np.mean(best_thresholds)
print(f"\n{'='*70}")
print(f"최종 권장 임계값: {optimal_threshold_final:.3f}")
print(f"이 값을 test 데이터 예측 시 사용하세요.")
print(f"{'='*70}")

print("\n✓ 단계 4 완료")

# ============================================================
# 5. 추가 분석: 이전 제출 결과와 비교
# ============================================================

print("\n" + "=" * 70)
print("이전 제출 결과 비교 (참고)")
print("=" * 70)

try:
    submission_old = pd.read_csv('submission_detailed_1.csv')
    print(f"이전 제출:")
    print(f"  평균 Confidence: {submission_old['confidence'].mean():.4f}")
    print(f"  낮은 Confidence (<0.1): {(submission_old['confidence'] < 0.1).sum()}개 "
          f"({(submission_old['confidence'] < 0.1).sum()/len(submission_old)*100:.1f}%)")
    print(f"\n개선 기대:")
    print(f"  평균 Confidence: {np.mean([r['avg_confidence'] for r in fold_results]):.4f}")
    print(f"  낮은 Confidence (<0.1): 평균 {np.mean([r['low_confidence_pct'] for r in fold_results]):.1f}%")
except FileNotFoundError:
    print("이전 제출 파일을 찾을 수 없습니다.")

print("\n" + "=" * 70)


단계 4: 피처 선택 기반 임계값 최적화

[피처 선택]
  전체 피처: 3076개
  선택된 피처: 150개
  제거된 피처: 2926개
  선택된 물성 피처: ['clogp', 'sa_score', 'qed', 'MolWt']

데이터 크기: (8349, 150)
레이블 분포: {1: 4542, 0: 3807}

전처리 대상:
  Fingerprint: 146개
  물성: 4개

교차검증 시작


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



[Fold 1]
피처: 150개 사용
학습 데이터: (6679, 150), 검증 데이터: (1670, 150)

[성능 비교]
  기본 임계값 (0.50): F1 = 0.8257
  최적 임계값 (0.400): F1 = 0.8398
  개선량: +0.0140

[추가 메트릭]
  ROC-AUC: 0.8904
  PR-AUC: 0.9014
  평균 Confidence: 0.3046
  낮은 Confidence (<0.1): 226개 (13.5%)

[혼동행렬 - 최적 임계값]
[[555 206]
 [102 807]]
  TN=555, FP=206, FN=102, TP=807
  FPR=0.271, FNR=0.112

[분류 리포트 - 최적 임계값]
              precision    recall  f1-score   support

      비활성(0)     0.8447    0.7293    0.7828       761
       활성(1)     0.7966    0.8878    0.8398       909

    accuracy                         0.8156      1670
   macro avg     0.8207    0.8085    0.8113      1670
weighted avg     0.8186    0.8156    0.8138      1670



/tmp/ipython-input-38380234.py:245: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-38380234.py:246: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missing from font(s) DejaVu Sans.
  plt.savefig('threshold_optimization_analysis.png', dpi=300, bbox_inches='tight')



✓ 시각화 저장: threshold_optimization_analysis.png


...


[Fold 3]
피처: 150개 사용
학습 데이터: (6679, 150), 검증 데이터: (1670, 150)

[성능 비교]
  기본 임계값 (0.50): F1 = 0.7815
  최적 임계값 (0.300): F1 = 0.8002
  개선량: +0.0187

[추가 메트릭]
  ROC-AUC: 0.8559
  PR-AUC: 0.8669
  평균 Confidence: 0.2977
  낮은 Confidence (<0.1): 245개 (14.7%)

[혼동행렬 - 최적 임계값]
[[475 287]
 [111 797]]
  TN=475, FP=287, FN=111, TP=797
  FPR=0.377, FNR=0.122

[분류 리포트 - 최적 임계값]
              precision    recall  f1-score   support

      비활성(0)     0.8106    0.6234    0.7047       762
       활성(1)     0.7352    0.8778    0.8002       908

    accuracy                         0.7617      1670
   macro avg     0.7729    0.7506    0.7525      1670
weighted avg     0.7696    0.7617    0.7566      1670



/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



[Fold 4]
피처: 150개 사용
학습 데이터: (6679, 150), 검증 데이터: (1670, 150)

[성능 비교]
  기본 임계값 (0.50): F1 = 0.8046
  최적 임계값 (0.370): F1 = 0.8244
  개선량: +0.0198

[추가 메트릭]
  ROC-AUC: 0.8808
  PR-AUC: 0.8934
  평균 Confidence: 0.2982
  낮은 Confidence (<0.1): 225개 (13.5%)

[혼동행렬 - 최적 임계값]
[[515 247]
 [ 98 810]]
  TN=515, FP=247, FN=98, TP=810
  FPR=0.324, FNR=0.108

[분류 리포트 - 최적 임계값]
              precision    recall  f1-score   support

      비활성(0)     0.8401    0.6759    0.7491       762
       활성(1)     0.7663    0.8921    0.8244       908

    accuracy                         0.7934      1670
   macro avg     0.8032    0.7840    0.7868      1670
weighted avg     0.8000    0.7934    0.7901      1670



/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



[Fold 5]
피처: 150개 사용
학습 데이터: (6680, 150), 검증 데이터: (1669, 150)

[성능 비교]
  기본 임계값 (0.50): F1 = 0.8165
  최적 임계값 (0.390): F1 = 0.8287
  개선량: +0.0122

[추가 메트릭]
  ROC-AUC: 0.8887
  PR-AUC: 0.8978
  평균 Confidence: 0.2973
  낮은 Confidence (<0.1): 235개 (14.1%)

[혼동행렬 - 최적 임계값]
[[541 220]
 [110 798]]
  TN=541, FP=220, FN=110, TP=798
  FPR=0.289, FNR=0.121

[분류 리포트 - 최적 임계값]
              precision    recall  f1-score   support

      비활성(0)     0.8310    0.7109    0.7663       761
       활성(1)     0.7839    0.8789    0.8287       908

    accuracy                         0.8023      1669
   macro avg     0.8075    0.7949    0.7975      1669
weighted avg     0.8054    0.8023    0.8002      1669


최종 통계

[피처 선택 효과]
  사용 피처: 150개
  제거 피처: 2926개

[기본 임계값 (0.5)]
  평균 F1: 0.8073
  표준편차: 0.0148
  Fold별: ['0.8257', '0.8084', '0.7815', '0.8046', '0.8165']

[최적화된 임계값]
  평균 F1: 0.8226
  표준편차: 0.0130
  Fold별: ['0.8398', '0.8199', '0.8002', '0.8244', '0.8287']

[임계값 통계]
  평균 임계값: 0.376
  표준편차: 0.041
  범위: [0

In [12]:
# ============================================================
# 단계 5: 최종 모델 학습 및 테스트 예측 (개선 버전)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

print("\n" + "=" * 70)
print("단계 5: 최종 모델 학습 및 테스트 예측")
print("=" * 70)

# ============================================================
# 5.1 피처 선택 (3단계 결과 활용)
# ============================================================

print("\n[5.1] 피처 선택...")

# 특징 중요도 로드
importance_df = pd.read_csv('feature_importance_cv.csv')

# 상위 150개 피처 선택 (4단계에서 사용한 것과 동일)
TOP_N = 150
selected_features = importance_df.head(TOP_N)['feature'].tolist()

print(f"  전체 피처: {len(importance_df)}개")
print(f"  선택된 피처: {len(selected_features)}개")
print(f"  제거된 피처: {len(importance_df) - len(selected_features)}개")

# 물성 피처 확인
desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']
desc_in_selected = [f for f in selected_features if f in desc_cols]
print(f"  물성 피처: {desc_in_selected}")

# ============================================================
# 5.2 최적 임계값 로드 (4단계 결과 활용)
# ============================================================

print("\n[5.2] 최적 임계값 로드...")

# 4단계 결과 로드
threshold_results = pd.read_csv('threshold_optimization_results_v2.csv')
optimal_threshold = threshold_results['threshold'].mean()

print(f"  Fold별 임계값: {threshold_results['threshold'].tolist()}")
print(f"  평균 임계값: {optimal_threshold:.3f}")
print(f"  표준편차: {threshold_results['threshold'].std():.3f}")
print(f"  범위: [{threshold_results['threshold'].min():.3f}, {threshold_results['threshold'].max():.3f}]")

# 최종 임계값 결정 (보수적으로 평균값 사용)
FINAL_THRESHOLD = optimal_threshold
print(f"\n  ✓ 최종 사용 임계값: {FINAL_THRESHOLD:.3f}")

# ============================================================
# 5.3 전체 학습 데이터로 최종 모델 학습
# ============================================================

print("\n[5.3] 전체 학습 데이터로 최종 모델 학습...")

# 학습 데이터 로드 (선택된 피처만 사용)
df_train = pd.read_csv('train.csv')
X_train_full = df_train[selected_features]
y_train_full = df_train['label'].astype(int)

print(f"  학습 데이터: {X_train_full.shape}")
print(f"  레이블 분포: {y_train_full.value_counts().to_dict()}")

# 전처리 파이프라인 구성 (선택된 피처 기반)
fp_cols_selected = [f for f in selected_features if f.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
desc_cols_selected = [f for f in selected_features if f in desc_cols]

preprocessor_final = ColumnTransformer(
    transformers=[
        ('fp', SimpleImputer(strategy='constant', fill_value=0), fp_cols_selected),
        ('desc', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), desc_cols_selected)
    ],
    remainder='drop'
)

# 전처리 적용
Xt_train_full = preprocessor_final.fit_transform(X_train_full)
print(f"  전처리 후: {Xt_train_full.shape}")

# 최종 모델 학습 (4단계에서 사용한 개선된 하이퍼파라미터)
final_model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1.0,
    reg_lambda=1.0,
    class_weight='balanced',
    random_state=42,
    verbose=-1
)

final_model.fit(Xt_train_full, y_train_full)
print(f"  ✓ 학습 완료: {Xt_train_full.shape[0]:,}개 샘플")

# ============================================================
# 5.4 테스트 데이터 로드 및 전처리
# ============================================================

print("\n[5.4] 테스트 데이터 전처리...")

df_test = pd.read_csv('predict_input.csv')
print(f"  테스트 데이터: {df_test.shape}")

# SMILES 컬럼 저장 (나중에 제출 파일에 사용)
test_smiles = df_test['SMILES'].copy()

# 선택된 피처만 추출
X_test = df_test[selected_features]
print(f"  선택된 피처 추출: {X_test.shape}")

# 전처리 적용 (학습 데이터와 동일한 변환)
Xt_test = preprocessor_final.transform(X_test)
print(f"  ✓ 전처리 완료: {Xt_test.shape}")

# ============================================================
# 5.5 확률 예측 및 분석
# ============================================================

print("\n[5.5] 확률 예측...")

y_test_proba = final_model.predict_proba(Xt_test)[:, 1]

print(f"  확률 통계:")
print(f"    평균: {y_test_proba.mean():.4f}")
print(f"    중앙값: {np.median(y_test_proba):.4f}")
print(f"    표준편차: {y_test_proba.std():.4f}")
print(f"    범위: [{y_test_proba.min():.4f}, {y_test_proba.max():.4f}]")

# 확률 분포 확인
prob_bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
prob_hist, _ = np.histogram(y_test_proba, bins=prob_bins)
print(f"\n  확률 분포:")
for i in range(len(prob_bins)-1):
    count = prob_hist[i]
    pct = count / len(y_test_proba) * 100
    print(f"    [{prob_bins[i]:.1f}-{prob_bins[i+1]:.1f}): {count:4d}개 ({pct:5.1f}%)")

# ============================================================
# 5.6 최적 임계값 적용 및 예측
# ============================================================

print(f"\n[5.6] 최적 임계값 ({FINAL_THRESHOLD:.3f}) 적용...")

y_test_pred = (y_test_proba >= FINAL_THRESHOLD).astype(int)

print(f"  예측 레이블 분포:")
print(f"    독성 有(0): {(y_test_pred == 0).sum():4d}개 ({(y_test_pred == 0).sum() / len(y_test_pred) * 100:5.1f}%)")
print(f"    독성 無(1):   {(y_test_pred == 1).sum():4d}개 ({(y_test_pred == 1).sum() / len(y_test_pred) * 100:5.1f}%)")

# ============================================================
# 5.7 Confidence 계산 및 분석
# ============================================================

print(f"\n[5.7] Confidence 분석...")

confidence = np.abs(y_test_proba - 0.5)

print(f"  Confidence 통계:")
print(f"    평균: {confidence.mean():.4f}")
print(f"    중앙값: {np.median(confidence):.4f}")
print(f"    표준편차: {confidence.std():.4f}")
print(f"    범위: [{confidence.min():.4f}, {confidence.max():.4f}]")

# Confidence 구간별 분포
conf_bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
conf_labels = ['매우 낮음 (0.0-0.1)', '낮음 (0.1-0.2)', '보통 (0.2-0.3)',
               '높음 (0.3-0.4)', '매우 높음 (0.4-0.5)']

print(f"\n  Confidence 구간별 분포:")
for i in range(len(conf_bins)-1):
    mask = (confidence >= conf_bins[i]) & (confidence < conf_bins[i+1])
    count = mask.sum()
    pct = count / len(confidence) * 100
    print(f"    {conf_labels[i]}: {count:4d}개 ({pct:5.1f}%)")

# 낮은 Confidence 샘플 식별
low_conf_mask = confidence < 0.1
low_conf_count = low_conf_mask.sum()
low_conf_pct = low_conf_count / len(confidence) * 100

print(f"\n  낮은 Confidence (<0.1) 샘플:")
print(f"    개수: {low_conf_count}개 ({low_conf_pct:.1f}%)")

if low_conf_count > 0:
    print(f"    확률 범위: [{y_test_proba[low_conf_mask].min():.4f}, {y_test_proba[low_conf_mask].max():.4f}]")
    print(f"    평균 확률: {y_test_proba[low_conf_mask].mean():.4f}")

# ============================================================
# 5.8 제출 파일 생성
# ============================================================

print(f"\n[5.8] 제출 파일 생성...")

# 기본 제출 파일
submission_basic = pd.DataFrame({
    'SMILES': test_smiles,
    'label': y_test_pred
})
submission_basic.to_csv('submission_final.csv', index=False)
print(f"  ✓ 기본 제출 파일 저장: submission_final.csv")

# 상세 제출 파일 (확률 및 confidence 포함)
submission_detailed = pd.DataFrame({
    'SMILES': test_smiles,
    'label': y_test_pred,
    'probability': y_test_proba,
    'confidence': confidence
})
submission_detailed.to_csv('submission_detailed_final.csv', index=False)
print(f"  ✓ 상세 제출 파일 저장: submission_detailed_final.csv")

# 낮은 confidence 샘플만 별도 저장 (추가 검토용)
if low_conf_count > 0:
    low_conf_samples = submission_detailed[low_conf_mask].copy()
    low_conf_samples = low_conf_samples.sort_values('confidence')
    low_conf_samples.to_csv('low_confidence_samples.csv', index=False)
    print(f"  ✓ 낮은 confidence 샘플 저장: low_confidence_samples.csv ({low_conf_count}개)")

# ============================================================
# 5.9 시각화
# ============================================================

print(f"\n[5.9] 결과 시각화...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1) 확률 분포
axes[0, 0].hist(y_test_proba, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=FINAL_THRESHOLD, color='r', linestyle='--',
                   label=f'Threshold={FINAL_THRESHOLD:.3f}')
axes[0, 0].axvline(x=0.5, color='gray', linestyle=':', alpha=0.5, label='Default (0.5)')
axes[0, 0].set_xlabel('Probability (Class 1)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Test Set Probability Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2) Confidence 분포
axes[0, 1].hist(confidence, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0.1, color='r', linestyle='--',
                   label=f'Low Confidence: {low_conf_count}개')
axes[0, 1].set_xlabel('Confidence (|prob - 0.5|)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Test Set Confidence Distribution')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3) 레이블 분포
label_counts = pd.Series(y_test_pred).value_counts().sort_index()
axes[1, 0].bar([0, 1], label_counts.values, color=['#ff7f0e', '#1f77b4'], alpha=0.7)
axes[1, 0].set_xticks([0, 1])
axes[1, 0].set_xticklabels(['toxic(0)', 'Nontoxic(1)'])
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Predicted Label Distribution')
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(label_counts.values):
    axes[1, 0].text(i, v + 10, f'{v}\n({v/len(y_test_pred)*100:.1f}%)',
                    ha='center', va='bottom')

# 4) 확률 vs Confidence Scatter
scatter = axes[1, 1].scatter(y_test_proba, confidence,
                             c=y_test_pred, cmap='coolwarm',
                             alpha=0.5, s=10)
axes[1, 1].axvline(x=FINAL_THRESHOLD, color='r', linestyle='--', alpha=0.7)
axes[1, 1].axhline(y=0.1, color='orange', linestyle='--', alpha=0.7)
axes[1, 1].set_xlabel('Probability (Class 1)')
axes[1, 1].set_ylabel('Confidence')
axes[1, 1].set_title('Probability vs Confidence')
axes[1, 1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1, 1], label='Predicted Label')

plt.tight_layout()
plt.savefig('test_prediction_analysis.png', dpi=300, bbox_inches='tight')
plt.close()

print(f"  ✓ 시각화 저장: test_prediction_analysis.png")

# ============================================================
# 5.10 최종 요약
# ============================================================

print("\n" + "=" * 70)
print("최종 요약")
print("=" * 70)

print(f"\n[모델 설정]")
print(f"  선택된 피처: {len(selected_features)}개")
print(f"  하이퍼파라미터:")
print(f"    n_estimators: 500")
print(f"    learning_rate: 0.05")
print(f"    max_depth: 7")
print(f"    정규화: L1=1.0, L2=1.0")

print(f"\n[임계값]")
print(f"  최종 임계값: {FINAL_THRESHOLD:.3f}")
print(f"  (CV 5-Fold 평균, 표준편차: {threshold_results['threshold'].std():.3f})")

print(f"\n[테스트 예측]")
print(f"  전체 샘플: {len(y_test_pred):,}개")
print(f"  독성 有(0): {(y_test_pred == 0).sum():,}개 ({(y_test_pred == 0).sum() / len(y_test_pred) * 100:.1f}%)")
print(f"  독성 無(1):   {(y_test_pred == 1).sum():,}개 ({(y_test_pred == 1).sum() / len(y_test_pred) * 100:.1f}%)")

print(f"\n[품질 지표]")
print(f"  평균 Confidence: {confidence.mean():.4f}")
print(f"  낮은 Confidence (<0.1): {low_conf_count}개 ({low_conf_pct:.1f}%)")

print(f"\n[교차검증 성능 (참고)]")
print(f"  평균 F1: {threshold_results['f1_optimized'].mean():.4f} ± {threshold_results['f1_optimized'].std():.4f}")
print(f"  평균 ROC-AUC: {threshold_results['roc_auc'].mean():.4f}")
print(f"  평균 PR-AUC: {threshold_results['pr_auc'].mean():.4f}")

print(f"\n[출력 파일]")
print(f"  ✓ submission_final.csv - 기본 제출 파일")
print(f"  ✓ submission_detailed_final.csv - 상세 정보 포함")
print(f"  ✓ test_prediction_analysis.png - 시각화")
if low_conf_count > 0:
    print(f"  ✓ low_confidence_samples.csv - 검토 필요 샘플")

print("\n" + "=" * 70)
print("✓ 단계 5 완료")
print("=" * 70)



단계 5: 최종 모델 학습 및 테스트 예측

[5.1] 피처 선택...
  전체 피처: 3076개
  선택된 피처: 150개
  제거된 피처: 2926개
  물성 피처: ['clogp', 'sa_score', 'qed', 'MolWt']

[5.2] 최적 임계값 로드...
  Fold별 임계값: [0.4, 0.42, 0.3, 0.37, 0.39]
  평균 임계값: 0.376
  표준편차: 0.046
  범위: [0.300, 0.420]

  ✓ 최종 사용 임계값: 0.376

[5.3] 전체 학습 데이터로 최종 모델 학습...
  학습 데이터: (8349, 150)
  레이블 분포: {1: 4542, 0: 3807}
  전처리 후: (8349, 150)
  ✓ 학습 완료: 8,349개 샘플

[5.4] 테스트 데이터 전처리...
  테스트 데이터: (927, 3077)
  선택된 피처 추출: (927, 150)
  ✓ 전처리 완료: (927, 150)

[5.5] 확률 예측...
  확률 통계:
    평균: 0.5218
    중앙값: 0.5213
    표준편차: 0.3351
    범위: [0.0044, 0.9993]

  확률 분포:
    [0.0-0.2):  232개 ( 25.0%)
    [0.2-0.4):  140개 ( 15.1%)
    [0.4-0.6):  133개 ( 14.3%)
    [0.6-0.8):  140개 ( 15.1%)
    [0.8-1.0):  282개 ( 30.4%)

[5.6] 최적 임계값 (0.376) 적용...
  예측 레이블 분포:
    독성 有(0):  363개 ( 39.2%)
    독성 無(1):    564개 ( 60.8%)

[5.7] Confidence 분석...
  Confidence 통계:
    평균: 0.3002
    중앙값: 0.3252
    표준편차: 0.1504
    범위: [0.0002, 0.4993]

  Confidence 구간별 분포:
    매우 낮음 (0.0-0.1):  1

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/tmp/ipython-input-2973936078.py:288: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-2973936078.py:289: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missing from font(s) DejaVu Sans.
  plt.savefig('test_prediction_analysis.png', dpi=300, bbox_inches='tight')


  ✓ 시각화 저장: test_prediction_analysis.png

최종 요약

[모델 설정]
  선택된 피처: 150개
  하이퍼파라미터:
    n_estimators: 500
    learning_rate: 0.05
    max_depth: 7
    정규화: L1=1.0, L2=1.0

[임계값]
  최종 임계값: 0.376
  (CV 5-Fold 평균, 표준편차: 0.046)

[테스트 예측]
  전체 샘플: 927개
  독성 有(0): 363개 (39.2%)
  독성 無(1):   564개 (60.8%)

[품질 지표]
  평균 Confidence: 0.3002
  낮은 Confidence (<0.1): 133개 (14.3%)

[교차검증 성능 (참고)]
  평균 F1: 0.8226 ± 0.0145
  평균 ROC-AUC: 0.8776
  평균 PR-AUC: 0.8892

[출력 파일]
  ✓ submission_final.csv - 기본 제출 파일
  ✓ submission_detailed_final.csv - 상세 정보 포함
  ✓ test_prediction_analysis.png - 시각화
  ✓ low_confidence_samples.csv - 검토 필요 샘플

✓ 단계 5 완료
